In [1]:
import sys, os
import pandas as pd
import setting

import sys, os
import pandas as pd
import setting
import pathSetting
sys.path.append("../../..")

from KETIToolDL.CLUSTTool.common import p1_integratedDataSaving as p1
from KETIToolDL.CLUSTTool.common import p2_dataSelection as p2
from KETIToolDL.CLUSTTool.common import p3_training as p3

import torch

#import main_regression as mr
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device}" " is available.")

cpu is available.


# 2. Training 

## 2-1. Data selection

In [2]:
dataRoot = 'ml_data_integration'
# 1 (p2부분 1. Data Selection)
DataMeta = p1.readJsonData(pathSetting.DataMetaPath)
dataList =  list(DataMeta.keys())

In [3]:
#2
# dataX
dataName_X = dataList[0]
dataSaveMode_X = DataMeta[dataName_X]["integrationInfo"]["DataSaveMode"]

# datay
dataName_y = dataList[1]
dataSaveMode_y = DataMeta[dataName_y]["integrationInfo"]["DataSaveMode"]

#3
dataX = p2.getSavedIntegratedData(dataSaveMode_X, dataName_X, dataRoot)
datay = p2.getSavedIntegratedData(dataSaveMode_y, dataName_y, dataRoot)
integration_freq_sec = DataMeta[dataName_X]["integrationInfo"]["integration_freq_sec"]

/home/keti/CLUST_KETI/Clust/KETIAppTestCode/JWTest/KUDataRegressionJWTest
/home/keti/CLUST_KETI/Clust/KETIAppTestCode/JWTest/KUDataRegressionJWTest


## 2-2. Training Data Preparation

In [4]:
# 2 Training Data Preparation
# 2-1
featureListX= list(dataX.columns)
featureListy= list(datay.columns)
target_col = 'value'

# 2-2
cleanTrainDataParam = 'NoClean'#  Classification, Regression과 같이 X, y가 분리된 경우에는 현재 고정해서 사용해야함

# 2-2-1 cleanTrainDataParam == Clean 일 경우
NaNProcessingParam ={
    "feature_cycle":'Day',
    "feature_cycle_times":1,
    "NanInfoForCleanData":{'type':'num', 'ConsecutiveNanLimit':3, 'totalNaNLimit':30000}
}
# 2-3
scalerParam='scale'

# 2-4
splitRatio = 0.8

# 2-5
scalerRootPath_X = os.path.join(pathSetting.scalerRootDir, dataName_X, cleanTrainDataParam)
scalerRootPath_y = os.path.join(pathSetting.scalerRootDir, dataName_X, cleanTrainDataParam)
train_x, val_x, X_scalerFilePath = p3.getTrainValData(dataX, featureListX, scalerRootPath_X, splitRatio, scalerParam)
train_y, val_y, y_scalerFilePath = p3.getTrainValData(datay, featureListy, scalerRootPath_y, splitRatio, scalerParam)

['RH_1', 'RH_2', 'RH_3', 'RH_4', 'RH_5', 'RH_6', 'RH_7', 'RH_8', 'RH_9', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'Press_mm_hg', 'RH_out', 'T_out', 'Tdewpoint', 'Visibility', 'Windspeed']
Make New scaler File
['value']
Make New scaler File


## 2-3 Training 

In [6]:
# 3-1.
model_list = ["LSTM_rg","GRU_rg", "CNN_1D_rg","LSTM_FCNs_rg"]
model_method = model_list[0]

#import main_regression as mr

n_epochs = 1000 # 학습 epoch 횟수, int(default: 1000, 범위: 1 이상)
batch_size = 16  # batch 크기, int(default: 16, 범위: 1 이상, 컴퓨터 사양에 적합하게 설정)

trainParameter = setting.modelTestconfig[model_method]
trainParameter['device']  = device

modelTags =["energy", "appliances", "regression"]
trainDataType = "timeseries"
modelPurpose = "regression"

# 2
trainDataInfo = DataMeta[dataName_X]['integrationInfo']

# 3. 모델을 저장할 파일 패스를 생성한다.
from KETIPreDataTransformation.general_transformation.dataScaler import encodeHashStyle
trainParameter_encode =  encodeHashStyle(str(trainParameter))
trainDataPathList = ["CLUST", "Electronics", dataName_X, trainParameter_encode]
modelFilePath = p3.getModelFilePath(trainDataPathList, model_method)

# 4. Training
from KETIToolDL.TrainTool.Regression.trainer import RegressionML as RML
rml = RML(model_method, trainParameter)
rml.processInputData(train_x, train_y, val_x, val_y, batch_size)
model = rml.getModel()
best_model = rml.trainModel(model, modelFilePath, n_epochs)

/home/keti/CLUST_KETI/Clust/KETIAppTestCode/JWTest/KUDataRegressionJWTest/../../../KETIPreDataTransformation/dataFormatTransformation/DFToNPArray.py:31: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  dfX_partial = dfX[startDate:endDate]
/home/keti/CLUST_KETI/Clust/KETIAppTestCode/JWTest/KUDataRegressionJWTest/../../../KETIPreDataTransformation/dataFormatTransformation/DFToNPArray.py:32: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  dfy_partial = dfy[startDate:endDate]
/home/keti/CLUST_KETI/Clust/KETIAppTestCode/JWTest/KUDataRegressionJWTest/../../../KETIPreDataTransformation/dataFormatTransformation/DFToNPArray.py:31: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated

Start training model

Epoch 1/1000
train Loss: 0.2104
val Loss: 0.1532

Epoch 10/1000
train Loss: 0.0747
val Loss: 0.0405

Epoch 20/1000
train Loss: 0.0424
val Loss: 0.0314

Epoch 30/1000
train Loss: 0.0415
val Loss: 0.0325

Epoch 40/1000
train Loss: 0.0407
val Loss: 0.0323

Epoch 50/1000
train Loss: 0.0397
val Loss: 0.0327

Epoch 60/1000
train Loss: 0.0383
val Loss: 0.0329

Epoch 70/1000
train Loss: 0.0363
val Loss: 0.0337

Epoch 80/1000
train Loss: 0.0340
val Loss: 0.0370

Epoch 90/1000
train Loss: 0.0306
val Loss: 0.0337

Epoch 100/1000
train Loss: 0.0282
val Loss: 0.0375

Epoch 110/1000
train Loss: 0.0291
val Loss: 0.0329

Epoch 120/1000
train Loss: 0.0243
val Loss: 0.0375

Epoch 130/1000
train Loss: 0.0217
val Loss: 0.0395

Epoch 140/1000
train Loss: 0.0221
val Loss: 0.0368

Epoch 150/1000
train Loss: 0.0157
val Loss: 0.0327

Epoch 160/1000
train Loss: 0.0149
val Loss: 0.0298

Epoch 170/1000
train Loss: 0.0134
val Loss: 0.0299

Epoch 180/1000
train Loss: 0.0127
val Loss: 0.0275

E

In [7]:
# 5. update MetaFile
ModelName = encodeHashStyle(p1.getListMerge ([trainParameter_encode, dataName_X, dataName_y] ))
modelInfoMeta ={
    "trainDataInfo":trainDataInfo,
    "featureList":featureListX,
    "target":featureListy,
    "trainDataType":trainDataType,
    "modelPurpose":modelPurpose,
    "model_method":model_method,
    "modelTags":modelTags,
    "cleanTrainDataParam":cleanTrainDataParam,
    "cleanTrainDataParam":NaNProcessingParam,
    "trainDataName":[dataName_X,dataName_y],
    "trainParameter":rml.parameter,
    "scalerParam":scalerParam,
    "X_scalerFilePath":X_scalerFilePath,
    "y_scalerFilePath":y_scalerFilePath,
    "modelFilePath":modelFilePath,
    "trainParameter":trainParameter
}
modelInfoMeta = p3.updateModelMetaData(ModelName, modelInfoMeta, pathSetting.trainModelMetaFilePath)